In [ ]:
# ============================================================
# 03_load_to_neo4j.ipynb
# Nạp dữ liệu từ Data Warehouse vào Neo4j (local)
# ============================================================

from pyspark.sql import SparkSession
from neo4j import GraphDatabase
from tqdm import tqdm
import pandas as pd
import os
import time

In [ ]:
# ------------------------------------------------------------
# 1️⃣ KHỞI TẠO SPARK SESSION
# ------------------------------------------------------------
spark = SparkSession.builder \
    .appName("MovieLens_Load_Neo4j") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

print("✅ SparkSession sẵn sàng.")

✅ SparkSession sẵn sàng.


In [ ]:
# ------------------------------------------------------------
# 2️⃣ KHAI BÁO KẾT NỐI NEO4J
# ------------------------------------------------------------

uri = "bolt://neo4j_db:7687"
user = "neo4j"
password = "mysecretpassword"   # mật khẩu ta đã thiết lập phần trước

driver = GraphDatabase.driver(uri, auth=(user, password))

with driver.session(database="movie") as session:
    result = session.run("RETURN '✅ Neo4j connected successfully!' AS msg")
    print(result.single()["msg"])

driver.close()

✅ Neo4j connected successfully!


In [15]:
# ------------------------------------------------------------
# 3️⃣ ĐỌC DỮ LIỆU TỪ DATA WAREHOUSE (PARQUET)
# ------------------------------------------------------------

WAREHOUSE_PATH = "../data/warehouse/"

# Load parquet
movies_df      = spark.read.parquet(WAREHOUSE_PATH + "dim_movies")
ratings_df     = spark.read.parquet(WAREHOUSE_PATH + "fact_ratings")
tags_df        = spark.read.parquet(WAREHOUSE_PATH + "dim_tags")
reviews_df     = spark.read.parquet(WAREHOUSE_PATH + "fact_reviews")
tag_count_df   = spark.read.parquet(WAREHOUSE_PATH + "fact_tag_count")
survey_df      = spark.read.parquet(WAREHOUSE_PATH + "fact_survey")

print("✅ Đã load dữ liệu từ Data Warehouse thành công!")

✅ Đã load dữ liệu từ Data Warehouse thành công!


In [16]:
print("\n📊 Bảng Fact & Dimension Preview:")
print(" ┣━ Movies:", movies_df.count())
print(" ┣━ Ratings:", ratings_df.count())
print(" ┣━ Tags:", tags_df.count())
print(" ┣━ Reviews:", reviews_df.count())
print(" ┣━ Tag Count:", tag_count_df.count())
print(" ┗━ Survey:", survey_df.count())


📊 Bảng Fact & Dimension Preview:
 ┣━ Movies: 84661
 ┣━ Ratings: 28249191
 ┣━ Tags: 1094
 ┣━ Reviews: 2628576
 ┣━ Tag Count: 212704
 ┗━ Survey: 51163


In [ ]:
movies_df.printSchema()
movies_df.show(5, truncate=False)

root
 |-- avg_rating: float (nullable = true)
 |-- dateAdded: string (nullable = true)
 |-- directed_by: string (nullable = true)
 |-- imdb_id: string (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- actors: string (nullable = true)
 |-- title: string (nullable = true)

+----------+---------+-----------------------------------------------------------------------+-------+-------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------+
|avg_rating|dateAdded|directed_by                                                            |imdb_id|item_id|actors                                                                                                                                                    

In [18]:
# ------------------------------------------------------------
# 4️⃣ CHUYỂN DỮ LIỆU SPARK → PANDAS VỚI LIMIT AN TOÀN
# ------------------------------------------------------------

LIMIT = 10000   # số dòng muốn lấy mẫu

movies_pd = movies_df.limit(LIMIT).toPandas()
ratings_pd = ratings_df.limit(LIMIT).toPandas()
tags_pd = tags_df.limit(LIMIT).toPandas()
reviews_pd = reviews_df.limit(LIMIT).toPandas()
tag_count_pd = tag_count_df.limit(LIMIT).toPandas()
survey_pd = survey_df.limit(LIMIT).toPandas()

print("✅ Đã chuyển mẫu dữ liệu sang Pandas (mỗi bảng 10,000 dòng).")

print(f"🎬 movies_pd: {len(movies_pd)} rows")
print(f"⭐ ratings_pd: {len(ratings_pd)} rows")
print(f"🏷 tags_pd: {len(tags_pd)} rows")
print(f"💬 reviews_pd: {len(reviews_pd)} rows")
print(f"🔢 tag_count_pd: {len(tag_count_pd)} rows")
print(f"📝 survey_pd: {len(survey_pd)} rows")

✅ Đã chuyển mẫu dữ liệu sang Pandas (mỗi bảng 10,000 dòng).
🎬 movies_pd: 10000 rows
⭐ ratings_pd: 10000 rows
🏷 tags_pd: 1094 rows
💬 reviews_pd: 10000 rows
🔢 tag_count_pd: 10000 rows
📝 survey_pd: 10000 rows


In [19]:
# ============================================================
# 5️⃣ NẠP DỮ LIỆU VÀO NEO4J (TỐI ƯU BATCH - UNWIND)
# ============================================================

from neo4j import GraphDatabase
from tqdm.notebook import tqdm
import time

# ------------------------------------------------------------
# 5.1 KẾT NỐI NEO4J
# ------------------------------------------------------------
uri = "bolt://neo4j_db:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "mysecretpassword"))
print("✅ Kết nối Neo4j thành công.")


# ------------------------------------------------------------
# 5.2 RESET DB + TẠO CONSTRAINTS
# ------------------------------------------------------------
def reset_and_prepare(driver):
    with driver.session(database="movie") as session:
        print("🧹 Đang xóa dữ liệu cũ...")
        session.run("MATCH (n) DETACH DELETE n")

        print("🛡️ Đang tạo constraints...")
        session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (m:Movie) REQUIRE m.item_id IS UNIQUE")
        session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (t:Tag) REQUIRE t.tag_id IS UNIQUE")
        session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (u:User) REQUIRE u.user_id IS UNIQUE")

    print("✅ DB đã sẵn sàng để nạp dữ liệu.\n")

reset_and_prepare(driver)


# ------------------------------------------------------------
# 5.3 HÀM LOAD THEO LÔ (BATCH)
# ------------------------------------------------------------
def run_batch(session, query, df, size=2000):
    records = df.to_dict("records")
    for i in tqdm(range(0, len(records), size), desc="⏳ Loading batch"):
        session.run(query, rows=records[i:i+size])


# ------------------------------------------------------------
# 5.4 BẮT ĐẦU NẠP DỮ LIỆU
# ------------------------------------------------------------
start_all = time.time()

with driver.session(database="movie") as session:

    # Movie
    print("\n🎬 Creating Movie nodes...")
    run_batch(session, """
        UNWIND $rows AS row
        MERGE (m:Movie {item_id: row.item_id})
        SET m.title = row.title,
            m.avg_rating = row.avg_rating,
            m.directed_by = row.directed_by,
            m.starring = row.actors,
            m.imdb_id = row.imdb_id,
            m.date_added = row.date_added
    """, movies_pd)

    # Tag
    print("\n🏷 Creating Tag nodes...")
    run_batch(session, """
        UNWIND $rows AS row
        MERGE (t:Tag {tag_id: row.tag_id})
        SET t.tag_name = row.tag_name
    """, tags_pd)

    # Movie -> Tag
    print("\n🔗 Creating HAS_TAG relationships...")
    run_batch(session, """
        UNWIND $rows AS row
        MATCH (m:Movie {item_id: row.item_id})
        MATCH (t:Tag {tag_id: row.tag_id})
        MERGE (m)-[r:HAS_TAG]->(t)
        SET r.num = row.num
    """, tag_count_pd)

    # User -> Movie
    print("\n⭐ Creating RATED relationships...")
    run_batch(session, """
        UNWIND $rows AS row
        MERGE (u:User {user_id: row.user_id})
        WITH u, row
        MATCH (m:Movie {item_id: row.item_id})
        MERGE (u)-[r:RATED]->(m)
        SET r.rating = row.rating
    """, ratings_pd)

    # Review node
    print("\n💬 Creating REVIEW nodes...")
    run_batch(session, """
        UNWIND $rows AS row
        MATCH (m:Movie {item_id: row.item_id})
        CREATE (r:Review {text: row.txt})
        MERGE (m)-[:HAS_REVIEW]->(r)
    """, reviews_pd)

driver.close()

print(f"\n🎉 HOÀN THÀNH! Tổng thời gian: {time.time() - start_all:.2f}s")

✅ Kết nối Neo4j thành công.
🧹 Đang xóa dữ liệu cũ...
🛡️ Đang tạo constraints...
✅ DB đã sẵn sàng để nạp dữ liệu.


🎬 Creating Movie nodes...


⏳ Loading batch:   0%|          | 0/5 [00:00<?, ?it/s]


🏷 Creating Tag nodes...


⏳ Loading batch:   0%|          | 0/1 [00:00<?, ?it/s]


🔗 Creating HAS_TAG relationships...


...


⭐ Creating RATED relationships...


⏳ Loading batch:   0%|          | 0/5 [00:00<?, ?it/s]


💬 Creating REVIEW nodes...


⏳ Loading batch:   0%|          | 0/5 [00:00<?, ?it/s]


🎉 HOÀN THÀNH! Tổng thời gian: 9.10s
